# Homework III &ensp;&ensp;&ensp;&ensp;       Experiments with Parsing
Matthew Stone, CS 533, Spring 2017
<p style="text-align:right">Jiaoying Tan</p>



Ops, there is no pynotebook this time so I edited some code and created this notebook. 

NOTE: I changed the *parser.py* to *_parser.py* in order to import it. Also changed *train-parser.py* to *train_parser.py* etc.

In [4]:
from train_parser import main as train_parser
from inspect_parser import main as inspect_parser

EN, en_train, en_dev = 'en', 'en-ud-trainpro.conllu', 'en-ud-devpro.conllu'
ZH, zh_train, zh_dev = 'zh', 'zh-ud-trainpro.conll', 'zh-ud-devpro.conll'

## 1. LEARNING EXPERIMENTS
First play with English data.

In [2]:
train_parser(EN, en_train, en_dev)

0 0.568
1 0.669
2 0.712
3 0.742
4 0.768
5 0.799
6 0.815
7 0.827
8 0.836
9 0.844
10 0.850
11 0.855
12 0.858
13 0.861
14 0.865
Averaging weights
Saving model to en\parser.pickle
Parsing took 6074.000 ms
17065 right of 20726 dependencies - accuracy: 0.823361960822


Inspect the parsing.

In [5]:
inspect_parser(EN, en_dev)

Parsing took 6220.000 ms
942 right of 1163 target moves accuracy: 0.809974
Features of class  A 7976 seen and  1166 new
Features of class  C 3936 seen and  694 new
Features of class  B 9142 seen and  0 new
Features of class  E 11620 seen and  10 new
Features of class  D 4047 seen and  5257 new
Features of class  F 11157 seen and  2799 new
Features of class  0 1163 seen and  0 new


Then we experiment with Chinese data

In [6]:
train_parser(ZH, zh_train, zh_dev)

0 0.449
1 0.563
2 0.629
3 0.682
4 0.718
5 0.771
6 0.796
7 0.816
8 0.834
9 0.848
10 0.855
11 0.868
12 0.875
13 0.880
14 0.884
Averaging weights
Saving model to zh\parser.pickle
Parsing took 3390.000 ms
7744 right of 10870 dependencies - accuracy: 0.71241950322


In [7]:
inspect_parser(ZH, zh_dev)

Parsing took 3405.000 ms
767 right of 933 target moves accuracy: 0.822079
Features of class  A 5552 seen and  1083 new
Features of class  C 2934 seen and  798 new
Features of class  B 6635 seen and  0 new
Features of class  E 9329 seen and  1 new
Features of class  D 3217 seen and  4247 new
Features of class  F 8723 seen and  2473 new
Features of class  0 933 seen and  0 new


## 2. DIAGNOSTIC EXPERIMENTS
First I modified *main* function in *inspect_parser.py*

In [9]:
def target_contexts(words, tags, i, n, stack, parse) :
    pass
def main(model_dir, heldout_gold, target=target_contexts):
    parser = Parser(load=True, model_dir=model_dir)
    c = 0
    t = 0
    gold_sents = list(read_ud_conll(heldout_gold))
    t1 = time.time()
    stats = Stats()
    stats.targetn = 0
    stats.correctn = 0
    stats.features_seen = {}
    stats.features_unseen = {}
    for (words, tags, gold_heads, gold_labels) in gold_sents:
        _, heads = parser.instrumented_parse(words, gold_heads, target, stats)
    t2 = time.time()
    print 'Parsing took %0.3f ms' % ((t2-t1)*1000.0)
    print stats.correctn, 'right of', stats.targetn, 'target moves',
    print '' if not stats.targetn else ('accuracy: %f' % (float(stats.correctn)/stats.targetn))
    for k in set(stats.features_seen.keys() + stats.features_unseen.keys()):
        print 'Features of class ', k,
        print '' if k not in stats.features_seen.keys() else stats.features_seen[k],
        print 'seen and ', 
        print '' if k not in stats.features_unseen.keys() else stats.features_unseen[k],
        print 'new'

Then define some patterns.

In [12]:
def target_p1(words, tags, i, n, stack, parse) :
    return len(stack) >=2 and tags[stack[-1]] == 'NOUN' and tags[stack[-2]] == 'NOUN' and tags[i] == 'NOUN'

def target_p2(words, tags, i, n, stack, parse) :
    return len(stack) >=1 and tags[stack[-1]] == 'NOUN' and tags[i] == 'VERB' 

Experiment with English data

In [13]:
inspect_parser(EN, en_dev, target_p1)

Parsing took 6557.000 ms
211 right of 299 target moves accuracy: 0.705686
Features of class  A 2212 seen and  384 new
Features of class  C 998 seen and  184 new
Features of class  B 2596 seen and  0 new
Features of class  E 2990 seen and  0 new
Features of class  D 1028 seen and  1364 new
Features of class  F 2777 seen and  811 new
Features of class  0 299 seen and  0 new


In [14]:
inspect_parser(EN, en_dev, target_p2)

Parsing took 6858.000 ms
899 right of 1124 target moves accuracy: 0.799822
Features of class  A 8079 seen and  1120 new
Features of class  C 3820 seen and  668 new
Features of class  B 9199 seen and  0 new
Features of class  E 11239 seen and  1 new
Features of class  D 4220 seen and  4772 new
Features of class  F 10954 seen and  2534 new
Features of class  0 1124 seen and  0 new


Experiment with Chinese data

In [15]:
inspect_parser(ZH, zh_dev, target_p1)

Parsing took 3541.000 ms
407 right of 453 target moves accuracy: 0.898455
Features of class  A 2677 seen and  533 new
Features of class  C 1499 seen and  313 new
Features of class  B 3210 seen and  0 new
Features of class  E 4530 seen and  0 new
Features of class  D 1842 seen and  1782 new
Features of class  F 4453 seen and  983 new
Features of class  0 453 seen and  0 new


In [16]:
inspect_parser(ZH, zh_dev, target_p2)

Parsing took 3590.000 ms
1052 right of 1371 target moves accuracy: 0.767323
Features of class  A 8845 seen and  1525 new
Features of class  C 4531 seen and  953 new
Features of class  B 10370 seen and  0 new
Features of class  E 13706 seen and  4 new
Features of class  D 5418 seen and  5550 new
Features of class  F 13514 seen and  2938 new
Features of class  0 1371 seen and  0 new


## 3.PROGRAMMING EXPERIMENTS
First I need to edit MOVES

In [18]:
SHIFT = 0; RIGHT = 1; LEFT = 2; REDUCE = 3;
MOVES = (SHIFT, RIGHT, LEFT, REDUCE)

then *transition* function

In [19]:
def transition(move, i, stack, parse):
    if move == SHIFT:
        stack.append(i)
        return i + 1
    elif move == RIGHT:
        parse.add(stack[-1], i)
        stack.append(i)
        return i + 1
    elif move == LEFT:
        parse.add(i, stack.pop())
        return i
    elif move == REDUCE:
        stack.pop()
        return i
    assert move in MOVES

then func *get_valid_moves*, notice the new function needs heads at current state.

In [20]:
def get_valid_moves(i, n, stack, heads):
    moves = []
    if (i+1) < n:
        moves.append(SHIFT)
    if len(stack) >= 1 and heads[stack[-1]] != None:
        moves.append(REDUCE)
    if len(stack) >= 1 and (i+1) < n:
        moves.append(RIGHT)
    if len(stack) >= 1 and heads[stack[-1]] == None:
        moves.append(LEFT)
    return moves

finally, func *get_gold_moves*

In [21]:
def get_gold_moves(n0, n, stack, heads, gold):
    valid = get_valid_moves(n0, n, stack, heads)
    costly = set([m for m in MOVES if m not in valid])
    if SHIFT not in costly:
        if gold[n0] in stack or any(heads[s] == None and gold[s] == n0 for s in stack):
            costly.add(SHIFT)
    if REDUCE not in costly:
        if len(stack) >= 1 and any(bh == stack[-1] for bh in gold[n0:]):
            costly.add(REDUCE)
    if RIGHT not in costly:
        if gold[n0] > n0 or any(heads[s] == None and gold[s] == n0 for s in stack):
            costly.add(RIGHT)
    if LEFT not in costly:
        if len(stack) >= 1 and (gold[stack[-1]] > n0 or any(bh == stack[-1] for bh in gold[n0+1:])):
            costly.add(LEFT)
    result = [m for m in MOVES if m not in costly]
    return result

Check performance on English data, modified *main* in *train_parser.py*

In [24]:
def main(model_dir, train_loc, heldout_gold, paser='_parser'):
    module = __import__(paser, globals(), locals(), ['*'])
    for k in dir(module):
        globals()[k] = getattr(module, k)
    pass

In [22]:
train_parser('en_eager', en_train, en_dev, paser='eager_parser')

Using arc eager parser
0 0.556
1 0.637
2 0.672
3 0.694
4 0.710
5 0.734
6 0.742
7 0.750
8 0.753
9 0.757
10 0.760
11 0.763
12 0.763
13 0.764
14 0.764
Averaging weights
Saving model to en_eager\parser.pickle
Parsing took 6675.000 ms
16712 right of 20726 dependencies - accuracy: 0.806330213259


And Chinese data

In [26]:
train_parser('zh_eager', zh_train, zh_dev, paser='eager_parser')

0 0.453
1 0.563
2 0.621
3 0.664
4 0.703
5 0.745
6 0.767
7 0.785
8 0.798
9 0.811
10 0.818
11 0.824
12 0.832
13 0.830
14 0.835
Averaging weights
Saving model to zh_eager\parser.pickle
Parsing took 3603.000 ms
7754 right of 10870 dependencies - accuracy: 0.713339466421
